<a href="https://colab.research.google.com/github/ekyuho/2024-1-iot/blob/main/whatsmatter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import json

api_key=''
url='http://175.197.63.177:8000/local/api/'

def run(t, method, h, j, p):
    headers={'x-api-key':api_key}
    headers.update(h)
    #print(url+t)
    if method=='get': r=requests.get(url+t, headers=headers, params=p, timeout=10)
    elif method=='post': r=requests.get(url+t, headers=headers, json=j, timeout=10)
    elif method=='put': r=requests.put(url+t, headers=headers, json=j, timeout=10)
    return r

In [3]:
print('1. 사용가능한 모든 디바이스 찾기')
r=run('devices', 'get', h={}, j={}, p={})
devices1=r.json()
print(f'got {len(devices1)} devices','\n')

rooms={}
services1={}
state1={}
for d in devices1:
    l,e=d['location'],d['entity']
    if l not in rooms: rooms[l]=[]
    rooms[l].append(e)

    r=run(f"devices/{d['entity']}/services", 'get', h={}, j={}, p={})
    services1[d['entity']]=r.json()

    if d['entity'].startswith('sensor') or d['entity'].startswith('binary_sensor'):
        data={'id': d['entity'], 'alias':d['entity'], 'trigger': {}}
        #r=run('notifications', 'put', h={}, j=data, p={})

print('2. 방별 사용가능 디바이스 상태: ', f'{len(rooms)}개의 방')
i=1
si=1
di=1
for r1 in rooms:
    print(f'방{i}', 'location=', r1)
    i+=1
    rooms[r1].sort()
    for e in rooms[r1]:
        if 'sensor' in e:
            name = f'센서{si}'
            si+=1
        else:
            name=f'디바이스{di}'
            di+=1
        print(f'      {name} entity=', e)
        r=run(f"states/{d['entity']}", 'get', h={}, j={}, p={})
        s=r.json()
        state1[d['entity']]=s
        print('          state=', s['state'])
print()

print('3. 전등/스위치 on/off')
for r1 in rooms:
    print(f'방{i}', 'location=', r1)
    for e in rooms[r1]:
        if 'light' in e or 'switch' in e:
            data={
                "domain": "light",
                "service" : "turn_on",
                "rgb_color" : [255,255,255]
            }
            r=run(f"devices/{e}/command", 'post', h={}, j=data, p={})  # 일단 light와 switch에 대해 tunn_on 하면 허용안된다는 405 코드리턴.
            print('     ',e, r)
print()

print('4. 센서별 trigger callback 지정')
for r1 in rooms:
    i=1
    for e in rooms[r1]:
        if 'sensor' in e:
            ####
            data={
                "id": e,
                "alias": e+str(i),
                "trigger": {
                  "state": "off",  # 센서(온도,습도,재실,문열림,조도) 사용을 위한  가이드 및 예시 필요.
                  "entity_id": e
                },
                "condition": [
                  {
                    "entity_id": "light.nano_vdevo",
                    "state": "off",
                    "option": "equal"
                  }
                ],
                "action": {
                  "url": 'http://20.39.195.245:8000/api/notify'
                  #  curl -H "Content-Type: application/json" --data '{"a":"b"}' http://20.39.195.245:8000/api/notify ==> {"result":"ok"}
                }
            }
            r=run(f"rules", 'post', h={}, j=data, p={})
            print(e, r)
        i+=1

1. 사용가능한 모든 디바이스 찾기
got 10 devices 

2. 방별 사용가능 디바이스 상태:  3개의 방
방1 location= test_room_id_1724216213468
      센서1 entity= binary_sensor.pir_sensor_occupancy
          state= off
      디바이스1 entity= light.nano_vdevo
          state= off
방2 location= test_room_id_1724226467853
      센서2 entity= binary_sensor.door_sensor_door
          state= off
      디바이스2 entity= light.zemismart_bulb_light
          state= off
      디바이스3 entity= switch.otopeulreiseu_kaendeulweomeo
          state= off
방3 location= test_room_id_1724320457362
      센서3 entity= binary_sensor.letianpai_presence_sensor_box_occupancy_2
          state= off
      센서4 entity= sensor.letianpai_presence_sensor_box_humidity_2
          state= off
      센서5 entity= sensor.letianpai_presence_sensor_box_illuminance_2
          state= off
      센서6 entity= sensor.letianpai_presence_sensor_box_temperature_2
          state= off
      디바이스4 entity= switch.smart_plug_in_unit_switch
          state= off

3. 전등/스위치 on/off
방4 location= te

In [ ]:
print(json.dumps(devices1,ensure_ascii=False, indent=4,default=str))

[
    {
        "alias": "라이트 나노 램프",
        "entity": "light.nano_vdevo",
        "location": "test_room_id_1724216213468"
    },
    {
        "alias": "캔들워머램프 김동은 프로 사무실",
        "entity": "switch.otopeulreiseu_kaendeulweomeo",
        "location": "test_room_id_1724226467853"
    },
    {
        "alias": "LetianPai 조도센서",
        "entity": "sensor.letianpai_presence_sensor_box_illuminance_2",
        "location": "test_room_id_1724320457362"
    },
    {
        "alias": "LetianPai 재실센서",
        "entity": "binary_sensor.letianpai_presence_sensor_box_occupancy_2",
        "location": "test_room_id_1724320457362"
    },
    {
        "alias": "LetianPai 습도센서",
        "entity": "sensor.letianpai_presence_sensor_box_humidity_2",
        "location": "test_room_id_1724320457362"
    },
    {
        "alias": "LetianPai 온도센서",
        "entity": "sensor.letianpai_presence_sensor_box_temperature_2",
        "location": "test_room_id_1724320457362"
    },
    {
        "alias": "Matter 플러

In [ ]:
print(json.dumps(services1,ensure_ascii=False, indent=4,default=str))

{
    "light.nano_vdevo": {
        "toggle": {
            "description": "Toggles one or more lights, from on to off, or, off to on, based on their current state.",
            "fields": {
                "advanced_fields": {
                    "collapsed": true,
                    "fields": {
                        "brightness": {
                            "filter": {
                                "attribute": {
                                    "supported_color_modes": [
                                        "brightness",
                                        "color_temp",
                                        "hs",
                                        "xy",
                                        "rgb",
                                        "rgbw",
                                        "rgbww"
                                    ]
                                }
                            },
                            "selector": {
                       